In [85]:
import scipy # python 3.8.10
from scipy import sparse
import numpy as np

In [86]:
# 
def generate_Gauss_local_1D(
        Gauss_coefficient_reference_1D, # 高斯积分点系数
        Gauss_point_reference_1D,
        lower_bound,upper_bound):
    Gauss_coefficient_local_1D=(upper_bound-lower_bound)*Gauss_coefficient_reference_1D/2
    # 这里用仿射变换
    Gauss_point_local_1D=(upper_bound-lower_bound)*Gauss_point_reference_1D/2+(upper_bound+lower_bound)/2

    return Gauss_coefficient_local_1D,Gauss_point_local_1D



In [87]:
def Gauss_quadrature_for_1D_integral_test(coefficient_function_name,Gauss_coefficient_local_1D,Gauss_point_local_1D,test_vertices,test_basis_type,test_basis_index,test_derivative_degree):

    Gpn=Gauss_coefficient_local_1D.shape()

    result=0
    for i in range(Gpn[0]):
        result=result+Gauss_coefficient_local_1D[i]*feval(coefficient_function_name,Gauss_point_local_1D[i])*local_basis_1D(Gauss_point_local_1D[i],test_vertices,test_basis_type,test_basis_index,test_derivative_degree)
    return result

利用1D积分下的矩阵组装

In [88]:
def assemble_matrix_from_1D_integral(
        coefficient_function_name, # 函数名称
        M_partition, # 存储了网格坐标
        T_partition, # 存储了每个单元上的节点的全局编号
        T_basis_trial, # 某个节点处的trial function
        T_basis_test, # 某个节点处的test function
        number_of_trial_local_basis, # 一个单元上的trial的数量
        number_of_test_local_basis, # 一个单元上的test的数量
        number_of_elements, # 单元的数量
        matrix_size, # 矩阵大小
        Gauss_coefficient_reference_1D, # 
        Gauss_point_reference_1D, 
        trial_basis_type,
        trial_derivative_degree,
        test_basis_type,
        test_derivative_degree):
    
    # coo矩阵比较好
    # result=sparse.coo_matrix

    # 无法像matlab一样初始化系数矩阵，需要先初始化一些数据
    row=[] # 记录行下标
    col=[] # 记录列下标
    data=[] # 记录数据结果

    # 对每个单元做循环
    for n in range(number_of_elements):

        # 获取全局节点编号
        vertices=M_partition[:,T_partition[:,n]]
        lower_bound=min[vertices[0],vertices[1]] # 获取左右(下)坐标
        upper_bound=max[vertices[0],vertices[1]]
        Gauss_coefficient_local_1D,Gauss_point_local_1D=generate_Gauss_local_1D(Gauss_coefficient_reference_1D,Gauss_point_reference_1D,lower_bound,upper_bound)
        
        for alpha in range(number_of_trial_local_basis):
            for beta in range(number_of_test_local_basis):   
                temp=Gauss_quadrature_for_1D_integral_trial_test(coefficient_function_name,Gauss_coefficient_local_1D,Gauss_point_local_1D,vertices,trial_basis_type,alpha,trial_derivative_degree,vertices,test_basis_type,beta,test_derivative_degree)
                # 记录下稀疏矩阵相关信息
                row.append(T_basis_test[beta,n])
                col.append(T_basis_trial[alpha,n])
                data.append(temp)

In [89]:

def generate_M_T_1D(left,right,h_partition,basis_type):
    # 
    h=h_partition #网格尺寸大小

    if basis_type==101:

       N=int((right-left)/h) # 网格个数
       M=np.zeros((1,N+1)) # 存储了网格坐标
       T=np.zeros((2,N),dtype=np.int32) # 存储了每个单元上的节点的全局编号

       for i in range(N+1):
           M[0,i]=left+i*h # 计算坐标       

       print("N",N)
       for i in range(N): # 计算得到每个单元节点的全局编号，类似 1---2  2---3 3---4 4---5 ... 这种
           T[0,i]=i
           T[1,i]=i+1
    
    elif basis_type==102: # 二阶单元

       N=int((right-left)/h)
       print("N",N)
       dh=h/2 # 由于单元中间还有个点，dh为h的一半
       dN=N*2 # 单元数加倍
       M=np.zeros((1,dN+1)) # 坐标数
       T=np.zeros((3,N)) # 全局有限元节点数量(不是网格节点)

       for i in range(dN+1):
           M[0,i]=left+i*dh       

       for i in range(N): # 全局节点编号， 类似 1--2--3 3--4--5 5--6--7 这种 
           T[0,i]=2*i+1     
           T[1,i]=2*i+3
           T[2,i]=2*(i+1)

    return M,T


In [90]:

def poisson_solver_1D(left,right,h_partition,basis_type,Gauss_point_number):
    # 单元数
    N_partition=(right-left)/h_partition

    # 根据单元类型，得到基函数个数
    if basis_type==102:
        N_basis=N_partition*2
    elif basis_type==101:
        N_basis=N_partition

    #Mesh information for partition and finite element basis functions.
    # 产生网格坐标与网格全局标号矩阵
    M_partition,T_partition=generate_M_T_1D(left,right,h_partition,101)
    print("M_partition\n",M_partition)
    print("T_partition\n",T_partition)

    # 若是二阶类型，需要重新生成有限元节点的(而非网格节点的)阵列
    if basis_type==102:
        M_basis,T_basis=generate_M_T_1D(left,right,h_partition,102)
        print("102",M_basis,"\n",T_basis)
    elif basis_type==101:
        M_basis=M_partition
        T_basis=T_partition


    # Guass quadrature's points and weights on the refenrece interval [-1,1].
    # 产生1D 单元的高斯点坐标与系数
    [   Gauss_coefficient_reference_1D,Gauss_point_reference_1D]=generate_Gauss_reference_1D(Gauss_point_number);

    # Assemble the stiffness matrix.
    number_of_elements=N_partition # 注意单元数与N_basis 的区别
    matrix_size=[N_basis+1,N_basis+1] #  N_basis是有限元节点的个数
    vector_size=N_basis+1
    if basis_type==102:
        number_of_trial_local_basis=3 #2阶的话每个单元三个有限元节点，所以 trial和test 都是3
        number_of_test_local_basis=3
    elif basis_type==101:
        number_of_trial_local_basis=2 # 同上，但是1阶只有两个有限元节点，trial 和test都是2
        number_of_test_local_basis=2

    # Assemble the stiffness matrix 组装刚度矩阵，因为是椭圆方程，所以trial和test都是一阶导数，二次元的一阶导数
    # A=assemble_matrix_from_1D_integral('function_a',M_partition,T_partition,T_basis,T_basis,number_of_trial_local_basis,number_of_test_local_basis,number_of_elements,matrix_size,Gauss_coefficient_reference_1D,Gauss_point_reference_1D,basis_type,1,basis_type,1);



    result=1
    return result

In [91]:
# def check_FE_solution_error_1D_quadratic():
#     xx


basis_type=102 # 使用二次单元求解
# %The problem domain is [left,right]*[bottom,top].
left=0
right=1


Gauss_point_number=4


h_partition=1/4
uh=poisson_solver_1D(left,right,h_partition,basis_type,Gauss_point_number)
print(uh)


N 4
M_partition
 [[0.   0.25 0.5  0.75 1.  ]]
T_partition
 [[0 1 2 3]
 [1 2 3 4]]
N 4
102 [[0.    0.125 0.25  0.375 0.5   0.625 0.75  0.875 1.   ]] 
 [[1. 3. 5. 7.]
 [3. 5. 7. 9.]
 [2. 4. 6. 8.]]
1
